In [1]:
import numpy as np
import pandas as pd

### 样例数据

In [2]:
dataset = pd.DataFrame({
    "ts": ["09-20"] * 3 + ["09-21"] * 3,
    "no": ["C9", "D5", "D3"] + ["C9", "D7", "D2"],
    "A1": [1, 3, 5] + [11, 13, 15],
    "B1": [6, 2, 7] + [16, 12, 17]
})
dataset

,ts,no,A1,B1
0,09-20,C9,1,6
1,09-20,D5,3,2
2,09-20,D3,5,7
3,09-21,C9,11,16
4,09-21,D7,13,12
5,09-21,D2,15,17



ts：时间维度

no：设备编号

A1：设备指标1

B1：设备指标2

需要按照时间维度压缩为一天一行数据，即09-20这一天下 C9_A1指标值为1 C9_B1指标值为6，D5_A1指标值为3，D5_B1指标值为2... 

如果每天出现的设备编号不能对其，则通过缺失值NaN填充，最后处理为一张稀疏表。

In [5]:
# 数据透视转换（两级列索引）
# 将相同日期的数据打平成一行，设置透视图的行索引（index）
# 根据相同设备号的数据组织透视图，设置透视图的列索引（columns）
# 设置透视指标列（values）
pivot_tb = dataset.pivot(index = 'ts', columns = 'no', values = ["A1", "B1"])

# 交换内外索引
pivot_tb = pivot_tb.swaplevel(axis = 1)

# 多级列索引排序
sorted_column_index = pivot_tb.columns.sort_values()

# 重制多级列索引
pivot_tb = pivot_tb.reindex(sorted_column_index, axis = 1)

# 多级列索引压缩为一级列索引（使用下划线连接）
pivot_tb.columns = pivot_tb.columns.map('_'.join)

# 重制单级列索引，并将数据透视图的行索引转换为一列
pivot_tb = pivot_tb.reset_index()
pivot_tb

,ts,C9_A1,C9_B1,D2_A1,D2_B1,D3_A1,D3_B1,D5_A1,D5_B1,D7_A1,D7_B1
0,09-20,1.0,6.0,NaN,NaN,5.0,7.0,3.0,2.0,NaN,NaN
1,09-21,11.0,16.0,15.0,17.0,NaN,NaN,NaN,NaN,13.0,12.0
